# data_analyze.ipynb

train 데이터셋의 모국어-카테고리 분포를 분석하고, 일정한 개수만 남겨 train/val/test 데이터셋 분할하는 파일

In [41]:
import pandas as pd
import matplotlib.pyplot as plt

In [42]:
merged_df = pd.read_pickle('./data/train/labeling/marged.pikl')

음성파일 길이 분석

In [43]:
merged_df.describe()

,recordTime,birthYear,LearningPeriod
count,587320.000000,587320.000000,587320.000000
mean,13.290898,1989.848185,49.362787
std,4.826729,9.573554,43.677255
min,1.437000,1955.000000,1.000000
25%,10.672000,1985.000000,24.000000
50%,12.347000,1993.000000,36.000000
75%,14.640000,1997.000000,60.000000
max,290.700000,2005.000000,360.000000


음성파일 8초 이상 14초 이하 데이터셋만 필터링

In [44]:
min_sec = 8
max_sec = 14
filtered_data = merged_df[(merged_df['recordTime'] >= min_sec) & (merged_df['recordTime'] <= max_sec)]
filtered_data = filtered_data.reset_index(drop=True)

In [45]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395023 entries, 0 to 395022
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   fileName          395023 non-null  object 
 1   speakerID         395023 non-null  object 
 2   sentenceID        395023 non-null  object 
 3   recordUnit        395023 non-null  object 
 4   recordQuality     395023 non-null  object 
 5   recordDate        395023 non-null  object 
 6   recordTime        395023 non-null  float64
 7   Reading           347074 non-null  object 
 8   ReadingLabelText  347074 non-null  object 
 9   Question          47949 non-null   object 
 10  AnswerLabelText   47949 non-null   object 
 11  SentenceSpeechLV  395023 non-null  object 
 12  SpeakerID         395023 non-null  object 
 13  gender            395023 non-null  object 
 14  birthYear         395023 non-null  int64  
 15  eduBackground     395023 non-null  object 
 16  country           39

In [46]:
filtered_data.columns

Index(['fileName', 'speakerID', 'sentenceID', 'recordUnit', 'recordQuality',
       'recordDate', 'recordTime', 'Reading', 'ReadingLabelText', 'Question',
       'AnswerLabelText', 'SentenceSpeechLV', 'SpeakerID', 'gender',
       'birthYear', 'eduBackground', 'country', 'residencePeriod',
       'residenceCity', 'languageClass', 'motherTongue', 'selfAssessment',
       'topikGrade', 'LearningPeriod', 'learningSource'],
      dtype='object')

In [47]:
import glob
import soundfile as sf
from tqdm import tqdm
import re

음성 파일 경로와 라벨 데이터로 활용할 transcribe text 가져와서 리턴하는 함수
- 음성파일에 이상이 있다면 None 리턴
- 각종 특수문자는 지우고 한글, 문장기호만 남김

In [48]:
def process_row(row):
    file_name = row['fileName']
    reading = row['ReadingLabelText']
    answer = row['AnswerLabelText']
    
    # 오디오 파일 찾기
    audio_files = glob.glob(f'./data/train/source/*/*/{file_name}')
    if not audio_files:
        return None  # 파일 없음

    audio_path = audio_files[0]
    
    # 오디오 파일이 정상인지 확인
    try:
        with sf.SoundFile(audio_path) as f:
            _ = f.frames
    except RuntimeError:
        return None  # 손상된 오디오

    # 텍스트 결정
    transcript = answer if pd.isna(reading) else reading
    
    pattern = r"[^가-힣a-zA-Z0-9\s?!\.]"
    cleaned_transcript = re.sub(pattern, "", transcript)
    
    return (audio_path, cleaned_transcript)

위에서 선언한 함수로 데이터 전처리

In [49]:
result = []
for i in tqdm(range(len(filtered_data))):
    r = process_row(filtered_data.iloc[i,:])
    if r is None: continue
    result.append((i, r[0], r[1]))

100%|██████████| 395023/395023 [01:30<00:00, 4343.54it/s]


In [50]:
result[0]

(0,
 './data/train/source/china/culture1/CN40QB226_CN0010_20210829.wav',
 '바쁜 직장 생활 때문에 쉬는 날에 집에 있게 되면 보통 맛있는 거 만들어서 어 시간을 보냅니다')

위에서 선택된 데이터셋만 필터링

In [51]:
filtered_data = filtered_data.iloc[[i[0] for i in result], :]

In [52]:
filtered_data = filtered_data.reset_index(drop=True)
filtered_data.shape

(365882, 25)

result 결과 입력

In [53]:
filtered_data['fileName'] = [i[1] for i in result]
filtered_data['type'] = [i[1].split('/')[5] for i in result]
filtered_data['text'] = [i[2] for i in result]
filtered_data.shape

(365882, 27)

필터링된 전체 데이터 저장

In [54]:
filtered_data.to_pickle('./data/train/labeling/filtered.pikl')

데이터 분포 확인

In [55]:
pd.crosstab(filtered_data['languageClass'], filtered_data['type'])

type,culture1,culture2,general,life1,life2
languageClass,,,,,
베트남어,18913,14345,21434,19242,17255
영어,1489,943,8512,1797,1905
일본어,27720,25192,25539,24629,24275
중국어,31587,0,38643,33311,29151


- 데이터가 하나도 존재하지 않는 부분이 있는 culture2 라벨 제외
- 모국어-카테고리 그룹화 한 데이터 중 각 항목마다 1489개 랜덤 샘플링

In [56]:
balanced_df = filtered_data[filtered_data['type'] != 'culture2'].groupby(['languageClass', 'type'], group_keys=False).apply(lambda x: x.sample(1489, random_state=42))
balanced_df.shape

/tmp/ipykernel_10285/3793741149.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = filtered_data[filtered_data['type'] != 'culture2'].groupby(['languageClass', 'type'], group_keys=False).apply(lambda x: x.sample(1489, random_state=42))


(23824, 27)

샘플링된 데이터 분포 확인

In [57]:
pd.crosstab(balanced_df['languageClass'], balanced_df['type'])

type,culture1,general,life1,life2
languageClass,,,,
베트남어,1489,1489,1489,1489
영어,1489,1489,1489,1489
일본어,1489,1489,1489,1489
중국어,1489,1489,1489,1489


샘플링된 데이터 저장

In [58]:
balanced_df.to_pickle('./data/train/labeling/filtered_balanced.pikl')

샘플링된 데이터를 제외하고 테스트 데이터 후보

In [59]:
filtered_data_test = filtered_data.drop(balanced_df.index.values)
filtered_data_test.shape

(342058, 27)

valid 후보 데이터 분포 확인

In [60]:
pd.crosstab(filtered_data_test['languageClass'], filtered_data_test['type'])

type,culture1,culture2,general,life1,life2
languageClass,,,,,
베트남어,17424,14345,19945,17753,15766
영어,0,943,7023,308,416
일본어,26231,25192,24050,23140,22786
중국어,30098,0,37154,31822,27662


- 데이터가 하나도 존재하지 않는 부분이 있는 culture2 라벨 제외
- 모국어-카테고리 그룹화 한 데이터 중 각 항목마다 308 랜덤 샘플링

In [72]:
balanced_df_test = filtered_data_test[filtered_data_test['type'] != 'culture2'].groupby(['languageClass', 'type'], group_keys=False).apply(lambda x: x.sample(308, random_state=42))
balanced_df_test.shape

/tmp/ipykernel_10285/996641020.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df_test = filtered_data_test[filtered_data_test['type'] != 'culture2'].groupby(['languageClass', 'type'], group_keys=False).apply(lambda x: x.sample(308, random_state=42))


(4620, 27)

필터링된 valid 데이터 분포 확인

In [73]:
pd.crosstab(balanced_df_test['languageClass'], balanced_df_test['type'])

type,culture1,general,life1,life2
languageClass,,,,
베트남어,308,308,308,308
영어,0,308,308,308
일본어,308,308,308,308
중국어,308,308,308,308


validation 데이터 저장

In [74]:
balanced_df_test.to_pickle('./data/train/labeling/filtered_balanced_valid.pikl')

train/valid 데이터 제외 후 test 데이터 후보로 남김

In [75]:
filtered_data_valid = filtered_data_test.drop(balanced_df_test.index.values)
filtered_data_valid.shape

(337438, 27)

분포 확인

In [76]:
pd.crosstab(filtered_data_valid['languageClass'], filtered_data_valid['type'])

type,culture1,culture2,general,life1,life2
languageClass,,,,,
베트남어,17116,14345,19637,17445,15458
영어,0,943,6715,0,108
일본어,25923,25192,23742,22832,22478
중국어,29790,0,36846,31514,27354


같은 방식으로 필터링

In [78]:
balanced_df_valid = filtered_data_valid[filtered_data_valid['type'] != 'culture2'].groupby(['languageClass', 'type'], group_keys=False).apply(lambda x: x.sample(108, random_state=42))
balanced_df_valid.shape

/tmp/ipykernel_10285/3309348974.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df_valid = filtered_data_valid[filtered_data_valid['type'] != 'culture2'].groupby(['languageClass', 'type'], group_keys=False).apply(lambda x: x.sample(108, random_state=42))


(1512, 27)

In [79]:
pd.crosstab(balanced_df_valid['languageClass'], balanced_df_valid['type'])

type,culture1,general,life1,life2
languageClass,,,,
베트남어,108,108,108,108
영어,0,108,0,108
일본어,108,108,108,108
중국어,108,108,108,108


테스트 데이터셋 저장

In [81]:
balanced_df_valid.to_pickle('./data/train/labeling/filtered_balanced_test.pikl')